# Digression ...student blogging ...  beginning of Lesson 3

Several people have been blogging or posting content for other to use  

#### Rashima  
    https://github.com/reshamas/fastai_deeplearn_part1  

#### Pavel Surmenok ... learning rate optimization notes  
    Deep dive, summary of notes, for learning rate optimization   
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0  

#### Radek ... A practioners guide to PyTorch
    A little more advanced. Numerical programming in general.  
    "https://towardsdatascience.com/a-practitioners-guide-to-pytorch-1d0f6a238040

#### Miguel ... Visualizing the relationship between Learning and Batch Size
    tried different batch sizes and learning rates  
    https://miguel-data-sc.github.io/2017-11-05-first/  


#### Samyam Bhutani  ... Convolutional Neural Network in 5 minutes
    https://hackernoon.com/convolutional-neural-network-in-5-minutes-8f867eb9ca39 


#### Anand Saha ... Decoding the ResNet architecture
    http://teleported.in/posts/decoding-resnet-architecture/

#### Apil Tamang ... Yet another ResNet Tutorial
    https://medium.com/@apiltamang/yet-another-resnet-tutorial-or-not-f6dd9515fcd7




### Course Structure

So far we did an intro to CNN's without theory or how they work. We have seen how to build a model that works exceptionally well. We will review some of that today. We will also begin to dig into some uderlying theory.

We will also intro application area: logistics, forecasting, financial data, language RNN, Collaborative filtering recommendations. Similar to what we did for images. Here's how you get state of the art result without theory. Then go back in reverse order dig into the theory and math and code underneath, first with collaborative filtering, structured data, cnn (images), an finally deep dive into recurrent neural networks.


<img src="./Lesson3-WhereWeGoFromHere.png" style='width:500px;height:500px;'>


### Some review from last time - dog breads


#### Download data from kaggle and/or anywhere else
 ##### Kaggle ... use Kaggle CLI. 
  - github ... floydwch/kaggle-cli
  - already in environment ... vim environment.yml
  - download from Kaggle website, if website changes it breaks so get recent version. 
  - pip install kaggle-cli --upgrade
  - Reshma Kaggle CLI Wiki 
  - kg download - username -p password -c competition_name  ... at Kaggle URL, after .../c /competition-name
  - make sure that ... download at least once to accept the rules ...need to signin with user password, not Google, may need to click forget password on kaggle ... ... 
  - will get folder with all competition data ... 
##### Maybe you don't want all the data.... for exampel  Planet data (kaggle) has Tiff (19 GB) and JPG (600 MB)
  - Here's a cool tip
  - chrome extension CurlWget
  - now when you try and download something start downloading in Chrome ... cancel it ... now go to yellow button (getCurl ... has cookies, login info itself) top right chrome ... copy and paste the command into paperspace terminal ... has all cookies and headers to dowload the file ... all stuff that is hidden behind a login ... VERY USEFUL for data science
  
##### where to put data
  - data is usually in a sub directory of notebook
  - courses dl1 folder, data is a symbolic link to a different drive
  - sim links are like aliases 
  - use ls -l flag
  


# Review the step by step process
start by turning data augmentation on ...  aug transforms = side on or top down ... 

For example, when we trained dogs and cats. The layers are important but the weights we need to adjust. The later ones need more trained. When we start with pre-trained. Last fully connecxted randomly set and this is what we initially trained. If like imagenet then this is all we need. When we unfreeze, we set learning rates for early layers set learning rate low because don't want to change much. For satellite this is no longer true and still need to adjust more relative to the dogs and cats situation. Online they talk about unfreezing subsets of layers. This approach of differential learning rates seems more effective than unfreezing subsets. You could skip this training just the last layer and go to differential but prob dont want to. The CNN layers are pretrainined, and at least better than nothing. However fully connected weights are fully random and need to do some training befgore unfreezing.

SGD will change weights on the kernals. The architecture will not change. 

Activations are calculated from the weights and labels (inputs), layer inputs. 

On satellite started with 64 x 64 ... How we get that 64 x 64 depends on transforms. By default, takes the smallest edge and zooms out and then takes the center crop. Data augmentation takes randomly chosen crop.

Test time augmentation also important.

We will learn about loss function next week. The metric is the thing that is printed. Interesting question could possibly train on single class data and apply to multiclass.

How do differential learning rates spread accross. There is concept of layer groups. In Fastai decides for you how to split. Last one fully connected. The others split approximately in halves. There are ways to overide. There are ways to dump the layers of the model. (learn.summary, spits out everything ... will get layer names and sizes, number of channels).  4 dimensional mini-batch .. number of images in minibatch dynamce, 3 channels 64 x 64 images 

Learning rate finder, if have tiny data set, need to make mini batches small. 

1. Set precompute=True
1. Find Learning rate ...  Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs ... pre-trainined image net normal pictures 
1. Turn off precompute ... Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer ... in this case 10x k ... when starting things with resnet then 10x if not at all like resnet images eg satallite images not like resnet then 3x will train longer for these type of images 
1. Use `lr_find()` again ... we didn't do this but we could to see if still looks the same from learning rate pserspective ... will print the learning rate of the last layer
1. Train full network with cycle_mult=2 until over-fitting


=>Lets do this again for another data set ... look at the lesson1-breeds.ipynb

## Entire process ( In a few frames) - Minimum Steps is shown below
  (skipped downloading data from kaggle)
``` 
   # import libraries 
   from fastai. conv_learner import *  # this library imports everything else
   PATH = "data/dogscats               # path where the data is ... 
   sz = 224; bs=64                     # image size nd batch size (images per batch)
                                       # sz = image size, initially may change this to smaller to run faster

   tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)  
                                               # transform data, side_on 
                                               # photos, zoom by 10%  
                                               # data is in paths dogs, a                                                                                      # paths cats ... you can rename this 
   data = ImageClassifierData.from_paths(PAT, tfms=tfms, bs=bs)  
                                               # from paths ... data, dog train,vaiid,test
                                               # fll in test name if going to submit to 
                                               # Kaggle
                                               
   learn = ConvLearner.pretrained(resnet50, data) # creae model from pre-trained ResNet50 model, with our data
   %time learn.fit(1e-2, 3, cycle_len=1)          
                                                   # call fit, 3 cycles of length 1
                                                   # by default, all but last few layers frozen, 
                                                   # takes 2.5 minutes
                                                   # could say precompute = True, a shortcut => faster
                                                   # data augmentation doesnt work if precompute=True
                                                   # learning rate 1 e-2
                                                   # 3 cycles of n cycles (cycle_len)
                                                   #   The number of epochs between resetting learning 
                                                   #   rate is set by cycle_len, and the number of times 
                                                   #   this happens is refered to as the number of cycles
                                                   # cycle_len =1 ... "cool trick" stochastic gradient 
                                                   #   descents with restart
                                                   #   cycle_len = 1, change learning rate after n cycles
                                                   #   as you get to the right spot. Start to decrease
                                                   #   learning rate
                                                   #   slow steps until get to the right spot. 
                                                   #   With more iterations
                                                   #  because activations are frozen
   learn.unfreeze() 
   learn.bn_freeze(True)                          
                                                   # bn_freeze larger data set similar imagenet side_on
                                                   #  similar objexts, 200 to 500 pixels ... then 
                                                   #  add this line.. batch norm moving avg's not updated
                                                   #  not supported by any other library
                                                   
   %time learn.fit([1e-5, 1e-4, 1e-2], 1, cycle_len=1)            # train again
   %time log_preds, y = learn.TTA()                               # test time augmentation to get best 
   metrics.log_loss(y, np.exp(log_preds)), accuracy(log_preds,y)  # see how well it did

```


# Jupyter Tricks that will be useful

- if not quite sure what method ... hit tab to get a list of methods
- shift tab to see arguements,  parameters to a method
- shift tab tab ... brings up the documentation
- shift tab tab tab ... separate window with all the information
- ?function ... will bring up documetation
- ??learn.predict ... pop up the source code
- Fastai method usually less 1/2 screen of code
- button on browser .. will bring up keybard shortcuts palette ... try and learn a few a day
- SHUTDOWN YOUR PAPERSPACE MACHINE ... same with Crestle ... 
- Go to forums and look at course.fast.ai for course information to get updates.

# Other libraries ... (Keras)
- Keras ... mainly sits on top of tensorFLow
- Keras lesson 1, parts of lesson 1 in Keras
- Keras also assumes ... validation set file, training set file 
- keras has many more things to set, if not correct all breakds
- rather than single data object ... have to create a data generater, with data augmentation, and normalization
- copy and past keras from internet is a good way to get started, not a set of 
- below cut and paste from keras documentation 

- SEE THE FILE keras_lesson1.ipynb and keras_lesson1_alg

### Review learning rate optimization here

#### Pavel Surmenok ... learning rate optimization notes  
    Deep dive, summary of notes, for learning rate optimization   
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0  

### Review also

>  Lesson1_alg  Cats and Dogs  
>  lesson1_breeds_alg

## Submitting to Kaggle (see lesson1-breed for more notes)

- Go to Kaggle website
  - for every competition there is a section called evalutaion and it will tell you what to submit
  - copied and pased a couple of lin

... copied first two lines, file format, below,  
... first line starts with "id" and csv list of dogbreeds  
... then, each line has the i-th id's followed by probability of dog breeds

```
   id,affenpinscher,afghan_hound,...,yorkshire_terrier
   000261fb3cbb32d8935728e486796680,0.0083,0,...,0.0083

```

So, how do we create that.   

Inside data object there is .classes (data.classes), alphabetical order of classes  

Inside **data.test_ds.fname** data set, there are all the filenames  

Dog breeds not provided in Keras style format, but instead CSV file of labels   
  ... use ImageClassiferData.from_csv (fastai) rather than ImageClassifierData.from_paths

from CSV file has classes (labels) data.classes

from data.test_ds.fnames we have the test images and filenames 

Good idea to use TTA

```
   log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
                                          # is_test = True ... predictions on test set rather than validation
                                          # by definition dont know labels for the test set
                                          # by default kaggle uses log so use exp to convert back to linear

   probs = np.mean(np.exp(log_preds),0)
   #accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels
   
   
   probs.shape # (n_images, n_classes)
     =>  (10357, 120)
     
   test set 10K images with 120 possible breeds ... need to convert it
   easiest way to do it is with pandas
     
   df = pd.DataFrame(probs)     # pass in matrix
   df.columns = data.classes    # name columns
   
   df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])   # insert col in position 0 that contains fnames
   
   notice filenames five letters 'test/123958672... .jpg' start we dont want and at end don't want
   so, subset
   now df should be correct
   
   next call df.to_csv ... save to file ... compress it, can be big ... ziped csv file on the servier
      wherever you are running 
      
   SUBM = f'{PATH}/subm/'
   os.makedirs(SUBM, exist_ok=True)
   df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)
   
   
   get it back to your computer, so you can double check and play
   or, use Kaggle CLI
   use FileLink
   
   FileLink(f'P{afds}subm.gz'}   ... with path on servier will give back URL and iwll download file on server
                                 URL cick on download file from server on your computer 
                                 will be saverd to downloads .... check it out and submit to Kaggle
                                 
   
   ```

